In [ ]:
import os
import sys
import pickle
import pandas as pd
from fuzzywuzzy import fuzz
import base64
import re
from email.mime.text import MIMEText
from googleapiclient.errors import HttpError
import ollama
from datetime import datetime
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from gmailoauth import authenticate_gmail_api
import gmailoauth  # Import the gmailoauth module
import time
import textwrap
from IPython.display import display, HTML, clear_output
import json
import threading
from IPython.display import display
import ipywidgets as widgets

sys.path.append("src")  # Add src directory to the path


# Authenticate start using gmailoauth.py
service = gmailoauth.authenticate_gmail_api()
print(service)

# printt



def printt(text):
    """
    Format text by wrapping paragraphs with a line width of 70 characters.
    
    :param text: The text to be formatted.
    :return: The formatted text string.
    """
    # Split the input text into paragraphs based on double newlines
    wrapped_paragraphs = [textwrap.fill(paragraph, width=70) for paragraph in text.split('\n\n')]
    
    # Join the wrapped paragraphs with triple newlines
    formatted_text = '\n\n'.join(wrapped_paragraphs)
    
    return formatted_text


def type_text_with_font(text, font="Arial", size=16, color="black", delay=0.1):
    """
    Simulate typing formatted text with custom font, size, and color in Jupyter Notebook.
    
    :param text: The text to display as typing.
    :param font: The font family to use (e.g., Arial, Courier, Times New Roman).
    :param size: The font size to use.
    :param color: The text color to use (e.g., black, red, blue).
    :param delay: The delay between each character (default is 0.1 seconds).
    """
    # Format the text using the printt function
    formatted_text = printt(text)
    
    styled_text = ""
    
    for char in formatted_text:
        styled_text += char  # Add each character to the string
        clear_output(wait=True)  # Clear the previous output
        # Display the current text with the specified font
        display(HTML(f'<p style="font-family:{font}; font-size:{size}px; color:{color}; white-space: pre-wrap;">{styled_text}</p>'))
        time.sleep(delay)   # Delay between each character

In [ ]:


# Function to write JSON data to file
def write_email_to_json(email_data, filename='emails.json'):
    try:
        with open(filename, 'a') as f:
            json.dump(email_data, f)
            f.write('\n')  # Separate entries by a newline for readability
    except Exception as e:
        print(f"Error writing to JSON: {e}")

# Function to read the last timestamp and mail_id from JSON file
def get_last_entry_from_json(filename='emails.json'):
    if not os.path.exists(filename):
        return None, None  # No file exists, no timestamp or mail_id to read
    
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            if lines:
                last_entry = json.loads(lines[-1])
                return last_entry.get('timestamp'), last_entry.get('mail_id')
    except Exception as e:
        print(f"Error reading from JSON: {e}")
    return None, None

# Fetch the first new email, skipping the last processed mail_id
def fetch_first_email(service, latest_timestamp, last_mail_id, filename='emails.json'):
    query = f"after:{int(latest_timestamp)}"
    results = service.users().messages().list(userId='me', q=query, labelIds=['INBOX']).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No new messages found.")
        return None

    # Retrieve messages and skip if they match the last mail_id
    for message in messages:
        if message['id'] == last_mail_id:
            # print("Skipping already processed email.")
            continue

        # Fetch the message
        msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
        
        # Prepare and write email data to JSON
        email_data = prepare_email_data(msg)
        write_email_to_json(email_data, filename=filename)
        print(f"Email with mail_id {email_data['mail_id']} stored as JSON.")
        
        return msg  # Return `msg` for further processing

    return None

# Prepare email data from `msg` for JSON storage
def prepare_email_data(msg):
    mail_id = msg['id']
    thread_id = msg['threadId']
    snippet = msg['snippet']
    headers = {header['name']: header['value'] for header in msg['payload']['headers']}
    sender = headers.get('From', 'Unknown')
    subject = headers.get('Subject', 'No Subject')
    timestamp = int(msg.get('internalDate', time.time())) // 1000
    labels = msg.get('labelIds', [])
    message_size = msg['sizeEstimate']
    payload = msg['payload']

    mimeType = None
    decoded_content = None

    if 'parts' in payload:
        for part in payload['parts']:
            if part.get('mimeType') == 'text/plain':
                decoded_content = base64.urlsafe_b64decode(part['body'].get('data', '')).decode('utf-8', errors='ignore')
                mimeType = part.get('mimeType')
                break

    attachments = []
    for part in payload.get('parts', []):
        if part.get('filename'):
            attachments.append(part['filename'])

    return {
        'mail_id': mail_id,
        'thread_id': thread_id,
        'labels': labels,
        'snippet': snippet,
        'sender': sender,
        'subject': subject,
        'timestamp': timestamp,
        'email_time': timestamp,
        'message_size': message_size,
        'payload': payload,
        'attachments': attachments,
        'headers': headers,
        'status': 'retrieved',
        'mimeType': mimeType,
        'Newest email content': decoded_content
    }

# Main loop to fetch emails, with option to use saved or start timestamp
def fetch_emails_with_timeout(service, interval_seconds, timeout_seconds, filename='emails.json', use_saved_timestamp=True):
    start_time = time.time()

    # Decide on latest timestamp based on user choice and get last mail_id
    latest_timestamp, last_mail_id = get_last_entry_from_json(filename) if use_saved_timestamp else (int(start_time), None)

    if latest_timestamp is None:
        latest_timestamp = int(start_time)

    # print(f"Using latest timestamp: {latest_timestamp} and last mail_id: {last_mail_id}")

    while True:
        
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout_seconds:
            print(f"{timeout_seconds} seconds elapsed, stopping fetch.")
            break

        # Fetch the latest email since the latest_timestamp, skip if same mail_id
        msg = fetch_first_email(service, latest_timestamp, last_mail_id, filename)

        # If a new email is received, update timestamp and mail_id
        if msg:
            latest_timestamp = int(msg.get('internalDate', time.time())) // 1000
            last_mail_id = msg['id']
            print("New message received, stopping fetch.")
            return msg  # Pass `msg` to other functions as needed

        time.sleep(interval_seconds)


In [ ]:
print(service)

In [ ]:


# Helper function to get sub-phrases of a specific length
def get_sub_phrases(text, phrase_length):
    words = text.split()
    return [' '.join(words[i:i+phrase_length]) for i in range(len(words) - phrase_length + 1)]

# Define the function to determine the email status based on routing rules
def route_email_status(msg, routing_memory_path='config/routing_memory.csv'):
    """
    Determine the status of an email based on routing rules from the routing memory CSV.

    Parameters:
    - msg: The email message object from Gmail API.
    - routing_memory_path: Path to the routing memory CSV file.

    Returns:
    - status: A string representing the determined status ('ignore', 'to_rep_notif', keyword, or None if no match).
    """

    # Load the routing memory from CSV
    rmdf = pd.read_csv(routing_memory_path)

    # Decode and retrieve necessary fields from msg
    sender = next(header['value'] for header in msg['payload']['headers'] if header['name'] == 'From')
    label_ids = msg.get('labelIds', [])

    # Decode the main content of the email (assume text/plain for simplicity)
    email_content = None
    if 'parts' in msg['payload']:
        for part in msg['payload']['parts']:
            if part.get('mimeType') == 'text/plain':
                email_content = base64.urlsafe_b64decode(part['body'].get('data', '')).decode('utf-8', errors='ignore')
                break

    # Ignore Rules: Check if any ignore conditions are met
    ignore_labels = [
        "CATEGORY_SOCIAL", "CATEGORY_PROMOTIONS", "CATEGORY_UPDATES",
        "CATEGORY_FORUMS", "SPAM"
    ]
    if any(label in label_ids for label in ignore_labels) or sender in rmdf['ignore'].values:
        return 'ignore'

    # Rule 1: Check if sender is in rmdf['important_contacts']
    if sender in rmdf['important_contacts'].values:
        return 'to_rep_notif'

    # Rule 2: Perform keyword phrase similarity matching for the email content
    if email_content:
        email_content_lower = email_content.lower()
        
        for keyword in rmdf['keywords'].dropna():
            keyword_lower = keyword.lower()
            keyword_length = len(keyword_lower.split())

            # Get all sub-phrases in the email content of the same word length as the keyword
            sub_phrases = get_sub_phrases(email_content_lower, keyword_length)
            
            for sub_phrase in sub_phrases:
                similarity_score = fuzz.partial_ratio(keyword_lower, sub_phrase)
                # print(sub_phrase , '   ', similarity_score)
                
                # Print and update status only when a match is found (similarity > 90)
                if similarity_score > 80:
                    print(f"Match found: '{keyword_lower}' with '{sub_phrase}' (Score: {similarity_score})")
                    return keyword  # Return the matched keyword as status

    # Return None if no rules apply
    return None


In [ ]:


def extract_main_body_and_history(email_content):
    """
    Extract the main body and thread history from the email content.

    Parameters:
    - email_content: The full email content as a string.

    Returns:
    - clean_body: The main body of the email as a string.
    - thread_history: The remaining email thread history as a string.
    """
    # Regex to detect common reply headers (e.g., "On DATE at TIME")
    reply_header_pattern = re.compile(
        r"(?i)(On\s+\w{3},\s+\w{3}\s+\d{1,2},\s+\d{4}\s+at\s+\d{1,2}:\d{2}\s*(AM|PM)?,?\s+.*wrote:)"
    )

    # Split the content based on the reply header pattern
    split_content = re.split(reply_header_pattern, email_content, maxsplit=1)
    
    # Main body is the text before the first reply header (if found)
    clean_body = split_content[0].strip()
    
    # Thread history is everything after the first reply header
    thread_history = ''.join(split_content[1:]).strip() if len(split_content) > 1 else ''

    return clean_body, thread_history

def get_email_content_and_extract(msg):
    """
    Decode the main content from `msg` and extract the main body and thread history.

    Parameters:
    - msg: The email message object from Gmail API.

    Returns:
    - clean_body: The main body of the email as a string.
    - thread_history: The remaining email thread history as a string.
    """
    # Decode the main content of the email (assume text/plain for simplicity)
    email_content = None
    if 'parts' in msg['payload']:
        for part in msg['payload']['parts']:
            if part.get('mimeType') == 'text/plain':
                email_content = base64.urlsafe_b64decode(part['body'].get('data', '')).decode('utf-8', errors='ignore')
                break

    if not email_content:
        print("No email content found.")
        return None, None

    # Extract main body and thread history from the email content
    clean_body, thread_history = extract_main_body_and_history(email_content)

    return clean_body, thread_history




In [ ]:


# Define the model and system prompt base for the chatbot
desiredModel = 'llama3.2:3b'

# Load domain knowledge from a file (e.g., 'readme.txt')
def load_domain_knowledge(file_path):
    """
    Loads domain knowledge from a specified text file.

    Parameters:
    - file_path: Path to the text file containing domain knowledge.

    Returns:
    - Maileed_How_It_Works: The content of the text file as a string.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            Maileed_How_It_Works = file.read().strip()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        Maileed_How_It_Works = "Domain knowledge file is missing."
    except UnicodeDecodeError:
        print(f"Unicode decode error for file: {file_path}. Trying alternative encoding.")
        try:
            with open(file_path, 'r', encoding='latin-1') as file:
                Maileed_How_It_Works = file.read().strip()
        except UnicodeDecodeError:
            print("Unable to decode the file with known encodings.")
            Maileed_How_It_Works = "Domain knowledge file contains unrecognized characters."
    return Maileed_How_It_Works


def format_main_body(clean_body, sender, email_time):
    """
    Format the main email body to include the sender information in the style of a reply.

    Parameters:
    - clean_body: The main body of the email as a string.
    - sender: The sender's email address.
    - email_time: The Unix timestamp of when the email was received.

    Returns:
    - formatted_body: The formatted main email body.
    """
    formatted_time = datetime.fromtimestamp(email_time).strftime('%a, %b %d, %Y at %I:%M %p')
    formatted_body = f"On {formatted_time} {sender} wrote:\n\n{clean_body}"
    return formatted_body

def generate_email_response(clean_body, thread_history, sender, email_time):
    """
    Generates a response to the main email body using Ollama 3.2, with thread history as context.

    Parameters:
    - clean_body: The main email body content.
    - thread_history: The email thread history as a string.
    - sender: The email address of the sender.
    - email_time: The Unix timestamp of when the email was received.

    Returns:
    - OllamaResponse: The generated response text.
    """
    # Format the main email body with sender information
    formatted_body = format_main_body(clean_body, sender, email_time)

    # Custom output format instructions
    out_format = '''
    Please output only a standard form of an email, starting with Salutation like 'Hi,', then email body, and finally your signature with your title. Nothing else.
    Your signature must be something like this: Best regards, Nas's Email Assistant AI Agent.
    If the agent name is misspelled in the email, such as 'mailead' or 'mailaed', you should use the correct spelling which is 'Maileed.'
    Never use misspelled forms instead of the correct spelling of Maileed.
    Don't forget to be focused on finding relevant information to each email from the Maileed_How_It_Works.
    Respond with absolute relevance to the main email body. When you are answering a question about Maileed, '''
   
    endofprompt = f'\n Just to remind you, now please, in only two paragraphs, respond to the last email as below: {formatted_body}'
    
    # Initialize the conversation history with the system prompt and context
    conversation_history = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': f'Email Body: {formatted_body}\n\nThread History:\n{thread_history}\n{endofprompt}  {out_format}'}
    ]

    # Call the Ollama model with the conversation history
    response = ollama.chat(model=desiredModel, messages=conversation_history)

    # Extract the model's response
    OllamaResponse = response['message']['content']
    return OllamaResponse


In [ ]:



def create_reply_message(sender, to, subject, message_text, thread_id, message_id):
    """
    Create a MIMEText email message with 'In-Reply-To' and 'References' headers for thread continuity.

    Parameters:
    - sender: The sender's email address.
    - to: Recipient's email address.
    - subject: Subject of the reply.
    - message_text: The email reply text.
    - thread_id: Thread ID of the email conversation.
    - message_id: Message ID of the original email.

    Returns:
    - A dictionary representing the raw MIME message and thread ID.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender  # Using "me" here to avoid the warning
    message['subject'] = f"Re: {subject}"
    message['In-Reply-To'] = message_id  # Use message_id directly
    message['References'] = message_id

    return {
        'raw': base64.urlsafe_b64encode(message.as_bytes()).decode(),
        'threadId': thread_id
    }

# Function to create a draft
def create_draft(service, user_id, message_body):
    """
    Create a draft message in the user's Gmail account.

    Parameters:
    - service: Gmail API service instance.
    - user_id: Email address or user ID ('me' for authenticated user).
    - message_body: The message dictionary containing raw MIME and threadId.

    Returns:
    - The draft object if successful; otherwise, None.
    """
    try:
        message = {'message': message_body}
        draft = service.users().drafts().create(userId=user_id, body=message).execute()
        print(f"[DEBUG] Draft created successfully. Draft ID: {draft['id']}")
        return draft
    except HttpError as error:
        print(f"[ERROR] An error occurred while creating the draft: {error}")
        return None

def send_draft(service, user_id, draft_id):
    """
    Send a draft message by its draft ID.

    Parameters:
    - service: Gmail API service instance.
    - user_id: Email address or user ID ('me' for authenticated user).
    - draft_id: ID of the draft to be sent.

    Returns:
    - The sent message object if successful; otherwise, None.
    """
    try:
        sent_message = service.users().drafts().send(userId=user_id, body={'id': draft_id}).execute()
        print(f"[DEBUG] Sent message successfully. Sent Message ID: {sent_message['id']}")
        return sent_message
    except HttpError as error:
        print(f"[ERROR] An error occurred while sending the draft: {error}")
        return None

def prepare_and_send_reply(msg, response, service, sender_email="me"):
    """
    Prepare an email reply draft using the generated response and send it if the recipient
    is in the approved addresses list.

    Parameters:
    - msg: The email message object from Gmail API.
    - response: The generated response text for the email body.
    - service: Gmail API service instance.

    Returns:
    - The sent message object if successful and recipient is approved; otherwise, None.
    """
    # Extract recipient, subject, thread ID, and message ID from msg
    recipient = next(header['value'] for header in msg['payload']['headers'] if header['name'] == 'From')
    subject = next(header['value'] for header in msg['payload']['headers'] if header['name'] == 'Subject')
    thread_id = msg['threadId']
    # Correctly extract the Message-ID header
    message_id = next((header['value'] for header in msg['payload']['headers'] if header['name'] == 'Message-ID'), None)

    if message_id is None:
        print("[ERROR] Original Message-ID not found. Cannot create a proper reply.")
        return None

    # Create the reply message
    message_body = create_reply_message(
        sender=sender_email,
        to=recipient,
        subject=subject,
        message_text=response,
        thread_id=thread_id,
        message_id=message_id
    )

    # Save the reply as a draft
    draft = create_draft(service, user_id='me', message_body=message_body)

    # Only send the draft if the recipient is in approved addresses
    if draft and recipient in approved_addresses:
        print("[INFO] Sending draft...")
        return send_draft(service, user_id='me', draft_id=draft['id'])
    else:
        print("[INFO] Draft created but not sent. Recipient not in approved addresses.")
        return draft  # Return the draft for reference


In [ ]:
# Config

desiredModel = 'llama3.2:3b'
# desiredModel = 'llama3.2:latest'

approved_addresses = ['name1 <emailaddress1@gmail.com>' , 'name2 <emailaddress2@gmail.com>']
senderaddress = 'myemail@gmail.com'



# Load domain knowledge from /readme.txt
Maileed_How_It_Works = load_domain_knowledge('readme.txt')

# Define the system prompt with the loaded domain knowledge
system_prompt = f'''
You are my email assistant for handling emails about 'Maileed AI Agent' and your title is User's Email Assistant."
Your main task is to find answers to any question in a given email solely based on the information in the domain knowledge below.
My name is <my_name> and my email address is <myemailaddress@gmail.com>.
All emails are about my new AI Agent called Maileed. Only respond to emails that are about Maileed. 
For any given email data, please keep track of the thread history to respond to the last email appropriately.
Pay attention to the date and time of each email in the thread to understand the timeline of the conversation.
Please respond in two paragraphs like a professional.

Response Rules:
If you find an answer in 'Maileed_How_It_Works' document, respond with that information accurately, as written.
If an answer is not found or is unclear based on 'Maileed_How_It_Works' document, you can respond that you will provide it as soon as you get further information late.
Prohibition on External Knowledge: Do not provide any information or make assumptions based on your general knowledge, external sources, or any information not explicitly stated in 'Maileed_How_It_Works' document. 

Here is the 'Maileed_How_It_Works' document specifically for your knowledge, it is not publically available. You should answer questions strictly based on the following Maileed_How_It_Works document:
{Maileed_How_It_Works}
'''






In [ ]:
# Initialize the stop flag
stop = False
# Define the stop button and its click event function
stop_button = widgets.Button(description="Stop Loop")
def on_button_click(b):
    global stop
    stop = True
# Attach the click event to the button
stop_button.on_click(on_button_click)
# Display the button in the Jupyter Notebook
display(stop_button)
# Define the loop function to run in a separate thread
def infinite_loop():
    global stop
    while not stop:
        print("Running...")

        msg = fetch_emails_with_timeout(service, interval_seconds=1, timeout_seconds=10, filename='emails.json', use_saved_timestamp=True)
        if msg:
            
            sender = next(header['value'] for header in msg['payload']['headers'] if header['name'] == 'From')
            print('________________sender:' , sender)

            if sender in approved_addresses:
                print('sender in approved addresses...')
                
                status = route_email_status(msg, routing_memory_path='config/routing_memory.csv')
    
    
                
                clean_body, thread_history = get_email_content_and_extract(msg)
        
                # Extract sender and email_time from msg
    
                email_time = int(msg.get('internalDate', time.time())) // 1000  # Convert to seconds if needed
                
                # Execute and generate the email response in one line
                response = generate_email_response(clean_body, thread_history, sender, email_time)
                
                # type_text_with_font(response, font="Arial", size=16, color="yellow", delay=0.01)
                print(response)
                
                prepare_and_send_reply(msg, response=response, service=service)


        
    
    # Print message after loop ends
    print("Loop stopped by user.")

# Start the loop in a separate thread
loop_thread = threading.Thread(target=infinite_loop)
loop_thread.start()


# Google Calendar

In [ ]:
# csf = os.path.join("config", "client_secret.json")

# SCOPES = [
#     'https://www.googleapis.com/auth/gmail.send',
#     'https://www.googleapis.com/auth/gmail.modify',
#     'https://www.googleapis.com/auth/gmail.compose',
#     'https://www.googleapis.com/auth/calendar'
# ]

# print(csf)


# creds = None
# token_file = os.path.join("config", "token.pickle")
# client_secret_file = csf  

# # Check if token.pickle file exists and load credentials from it.
# if os.path.exists(token_file):
#     with open(token_file, 'rb') as token:
#         creds = pickle.load(token)

# # If no valid credentials, initiate OAuth login process.
# if not creds or not creds.valid:
#     if creds and creds.expired and creds.refresh_token:
#         try:
#             creds.refresh(Request())
#         except Exception as e:
#             print(f"Token refresh failed: {e}")
#             # Delete the old token file and force re-authentication
#             if os.path.exists(token_file):
#                 os.remove(token_file)
#             creds = None
#     if not creds:
#         flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
#         creds = flow.run_local_server(port=0)
#         # Save the credentials for the next run
#         with open(token_file, 'wb') as token:
#             pickle.dump(creds, token)

# # Create the Gmail API service object
# print(creds)

In [ ]:
# service2 = build('calendar', 'v3', credentials=creds)

# # Fetch calendar events
# calendar_id = 'primary'  # or specify a different calendar ID
# events_result = service2.events().list(calendarId=calendar_id, maxResults=10, singleEvents=True, orderBy='startTime').execute()
# events = events_result.get('items', [])

# # Print event details
# for event in events:
#     print(event['summary'], event['start'].get('dateTime', event['start'].get('date')))